In [22]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [23]:
with open("17/example.txt") as f:
    data = f.read().splitlines()
graph = [
    [int(x) for x in row]
    for row in data
]

In [24]:
from collections import namedtuple
from enum import Enum

class Point(namedtuple("Point", ["y", "x"])):
    def __add__(self, other):
        return Point(self.y + other.y, self.x + other.x)
    
    def __sub__(self, other):
        return Point(self.y - other.y, self.x - other.x)


class Direction(Point, Enum):
    UP = Point(-1, 0)
    RIGHT = Point(0, 1)
    DOWN = Point(1, 0)
    LEFT = Point(0, -1)

    def turns(self):
        attr = "y" if self.y != 0 else "x"
        return [d for d in Direction if abs(getattr(self, attr)) != abs(getattr(d, attr))]


def possible_directions(last_direction: Direction, hops_in_this_direction: int):
    match last_direction, hops_in_this_direction:
        case _, 3:
            return [(turn, 1) for turn in last_direction.turns()]
        
        case _:
            return [(last_direction, hops_in_this_direction + 1), *[(t, 1) for t in last_direction.turns()]]

In [25]:
possible_directions(Direction.DOWN, 0)

[(<Direction.DOWN: Point(y=1, x=0)>, 1),
 (<Direction.RIGHT: Point(y=0, x=1)>, 1),
 (<Direction.LEFT: Point(y=0, x=-1)>, 1)]

In [26]:
# from functools import reduce
# from tqdm import tqdm

# def dijkstra(graph: list[list[int]], source: Point, target: Point):
#     dist = {}
#     prev = {}

#     queue = set()
#     starting_nodes = [(source, Direction.DOWN, 0), (source, Direction.RIGHT, 0)]
#     queue.add(starting_nodes[0])
#     # queue.add(starting_nodes[1])
#     for y in range(len(graph)):
#         for x in range(len(graph[0])):
#             point = Point(y, x)
#             for d in Direction:
#                 for hops in [1, 2, 3]:
#                     dist[(point, d, hops)] = float("inf")
#                     prev[(point, d, hops)] = None
#                     queue.add((point, d, hops))

#     dist[starting_nodes[0]] = 0
#     # dist[starting_nodes[1]] = 0

#     # print(dist)

#     def get_min():
#         min_node = None
#         min_dist = float("inf")
#         for node in queue:
#             if dist[node] < min_dist:
#                 min_dist = dist[node]
#                 min_node = node
#         return min_node, min_dist
    
#     with tqdm(total=target.x * target.y) as pbar:
#         max_y, max_x = 0, 0
#         while len(queue) > 0:
#             current_node, current_dist = get_min()
#             # print(current_node)
#             if current_node is None:
#                 break
#             point, direction, hops = current_node
#             y, x = point
#             if y > max_y and x > max_x:
#                 max_y = y
#                 max_x = x
#                 pbar.n = max_x * max_y 
#                 pbar.last_print_n = max_x * max_y
#                 pbar.refresh()
#             if point == target:
#                 break
#             queue.remove(current_node)
#             # print(current_node)

#             next_directions = possible_directions(direction, hops)
#             # print(next_directions)
#             new_points = [(point + new_direction, new_direction, new_hops) for (new_direction, new_hops) in next_directions]
#             # print(new_points)
            
#             for new_ in new_points:
#                 if new_ in queue:
#                     # print(new_)
#                     alt = dist[current_node] + graph[point.y][point.x]
#                     # print(alt)
#                     if alt < dist[new_]:
#                         dist[new_] = alt
#                         prev[new_] = current_node
#                     # break
#             # break
        
#         return dist, prev

In [27]:
from functools import reduce
from tqdm import tqdm

def dijkstra2(graph: list[list[int]], source: Point, target: Point):
    dist = {}
    prev = {}

    queue = set()
    queue_dict = {}
    queue_dict[float("inf")] = set()

    def add_to_queue(node, distance):
        queue.add(node)
        if distance not in queue_dict:
            queue_dict[distance] = set()
        queue_dict[distance].add(node)

    def remove_from_queue(node, distance):
        queue.remove(node)
        queue_dict[distance].remove(node)
        if len(queue_dict[distance]) == 0:
            queue_dict.pop(distance)


    for y in range(len(graph)):
        for x in range(len(graph[0])):
            point = Point(y, x)
            for d in Direction:
                for hops in [1, 2, 3]:
                    dist[(point, d, hops)] = float("inf")
                    prev[(point, d, hops)] = None

                    add_to_queue((point, d, hops), float("inf"))


    starting_node = (source, Direction.DOWN, 0)
    add_to_queue(starting_node, 0)
    dist[starting_node] = 0

    def get_min():
        min_dist = min(queue_dict.keys())
        min_node = next(iter(queue_dict[min_dist]))
        return min_node, min_dist
    
    with tqdm(total=target.x * target.y) as pbar:
        max_y, max_x = 0, 0
        while len(queue) > 0:
            current_node, current_dist = get_min()
            # print(current_node)
            if current_node is None:
                break
            point, direction, hops = current_node
            y, x = point
            if y > max_y and x > max_x:
                max_y = y
                max_x = x
                pbar.n = max_x * max_y 
                pbar.last_print_n = max_x * max_y
                pbar.refresh()
            if point == target:
                break
            remove_from_queue(current_node, current_dist)
            # print(current_node)

            next_directions = possible_directions(direction, hops)
            # print(next_directions)
            new_points = [(point + new_direction, new_direction, new_hops) for (new_direction, new_hops) in next_directions]
            # print(new_points)
            
            for new_ in new_points:
                if new_ in queue:
                    new_point_current_dist = dist[new_]
                    alt = dist[current_node] + graph[point.y][point.x]
                    if alt < dist[new_]:
                        dist[new_] = alt
                        prev[new_] = current_node
                        remove_from_queue(new_, new_point_current_dist)
                        add_to_queue(new_, alt)
                    # break
            # break
        
        return dist, prev

In [28]:
# from functools import reduce
# from tqdm import tqdm

# def dijkstra_simple(graph: list[list[int]], source: Point, target: Point):
#     dist = {}
#     prev = {}

#     queue = set()
#     for y in range(len(graph)):
#         for x in range(len(graph[0])):
#             point = Point(y, x)
#             dist[point] = float("inf")
#             prev[point] = None
#             queue.add(point)

#     dist[source] = 0

#     def get_min():
#         min_node = None
#         min_dist = float("inf")
#         for node in queue:
#             if dist[node] < min_dist:
#                 min_dist = dist[node]
#                 min_node = node
#         return min_node, min_dist
    
    
#     while len(queue) > 0:
#         point, current_dist = get_min()
#         if point == target:
#             break
#         queue.remove(point)

#         new_points = [point + d for d in Direction]
        
#         for new_ in new_points:
#             if new_ in queue:
#                 alt = dist[point] + graph[point.y][point.x]
                
#                 if alt < dist[new_]:
#                     dist[new_] = alt
#                     prev[new_] = point
#                 # break
#         # break
    
#     return dist, prev

In [29]:
source = Point(0,0)
target = Point(len(graph) - 1, len(graph[0]) - 1)
# target = Point(12, 12)
print(source, target)
dist, prev = dijkstra2(graph, source, target)

Point(y=0, x=0) Point(y=12, x=12)


 75%|███████▌  | 108/144 [00:00<00:00, 6622.88it/s]


In [30]:
# %lprun -f dijkstra2 dijkstra2(graph, source, target)

In [31]:
# source = Point(0,0)
# target = Point(len(graph) - 1, len(graph[0]) - 1)
# print(source, target)
# dist, prev = dijkstra_simple(graph, source, target)

In [32]:
# source = Point(0,0)
# target = Point(len(graph) - 1, len(graph[0]) - 1)
# print(source, target)
# dist, prev = dijkstra(graph, source, target)

In [33]:
# dist[Point(y=140, x=140)]

In [34]:
# dist

In [35]:
# [
#     d for d in dist.items() if d[0][0] == Point(12, 12)
# ]

In [36]:
s = []
target = Point(len(graph) - 1, len(graph[0]) - 1)
targets = [
    (n, d) for (n, d) in dist.items()
    if n[0] == target
]
target = reduce(lambda p1, p2: p1 if p1[1] < p2[1] else p2, targets)
u, min_dist = target
if prev[u] or u == Point(0, 0):
    while u:
        s.append(u)
        if u in prev:
            u = prev[u]
        else:
            u = None
s = s[::-1]
s

[(Point(y=0, x=0), <Direction.DOWN: Point(y=1, x=0)>, 0),
 (Point(y=0, x=1), <Direction.RIGHT: Point(y=0, x=1)>, 1),
 (Point(y=0, x=2), <Direction.RIGHT: Point(y=0, x=1)>, 2),
 (Point(y=1, x=2), <Direction.DOWN: Point(y=1, x=0)>, 1),
 (Point(y=1, x=3), <Direction.RIGHT: Point(y=0, x=1)>, 1),
 (Point(y=1, x=4), <Direction.RIGHT: Point(y=0, x=1)>, 2),
 (Point(y=1, x=5), <Direction.RIGHT: Point(y=0, x=1)>, 3),
 (Point(y=0, x=5), <Direction.UP: Point(y=-1, x=0)>, 1),
 (Point(y=0, x=6), <Direction.RIGHT: Point(y=0, x=1)>, 1),
 (Point(y=0, x=7), <Direction.RIGHT: Point(y=0, x=1)>, 2),
 (Point(y=0, x=8), <Direction.RIGHT: Point(y=0, x=1)>, 3),
 (Point(y=1, x=8), <Direction.DOWN: Point(y=1, x=0)>, 1),
 (Point(y=1, x=9), <Direction.RIGHT: Point(y=0, x=1)>, 1),
 (Point(y=2, x=9), <Direction.DOWN: Point(y=1, x=0)>, 1),
 (Point(y=2, x=10), <Direction.RIGHT: Point(y=0, x=1)>, 1),
 (Point(y=3, x=10), <Direction.DOWN: Point(y=1, x=0)>, 1),
 (Point(y=4, x=10), <Direction.DOWN: Point(y=1, x=0)>, 2),
 (

In [37]:
# s = []
# target = Point(len(graph) - 1, len(graph[0]) - 1)
# targets = [
#     (n, d) for (n, d) in dist.items()
#     if n[0] == target
# ]
# target = reduce(lambda p1, p2: p1 if p1[1] < p2[1] else p2, targets)
# u, min_dist = target
# if prev[u] or u == Point(0, 0):
#     while u:
#         s.append(u)
#         if u in prev:
#             u = prev[u]
#         else:
#             u = None
# s[::-1]

In [38]:
dir_map = {
    Direction.UP: "^",
    Direction.DOWN: "v",
    Direction.RIGHT: ">",
    Direction.LEFT: "<"
}

In [39]:
# from copy import deepcopy
# def print_graph_simple(graph, s = None):
#     graph = deepcopy(graph)
#     graph = ["".join([str(x) for x in row]) for row in graph]

#     if s:
#         for p in s:
#             y, x = p[0]
#             graph[y] = graph[y][:x] + "x" + graph[y][x+1:]
            
#     return graph

# print_graph_simple(graph, s)

In [40]:
from copy import deepcopy
def print_graph(graph, s = None):
    graph = deepcopy(graph)
    graph = ["".join([str(x) for x in row]) for row in graph]

    if s:
        for p, direction, _ in s:
            y, x = p
            graph[y] = graph[y][:x] + dir_map[direction] + graph[y][x+1:]
            
    return graph

print_graph(graph, s)

['v>>34^>>>1323',
 '32v>>>35v>623',
 '325524565v>54',
 '3446585845v52',
 '4546657867v>6',
 '14385987984v4',
 '44578769877v6',
 '36378779796v>',
 '465496798688v',
 '456467998645v',
 '12246868655<v',
 '25465488877v5',
 '43226746555v>']

In [41]:
graph[0][0]

2

In [42]:
rr = 0

for p, _, _ in s:
    y, x = p
    rr += graph[y][x]
rr - graph[0][0]

102

## Part 2

In [43]:
def possible_directions(last_direction: Direction, hops_in_this_direction: int):
    match last_direction, hops_in_this_direction:
        case _, 10:
            return [(turn, 1) for turn in last_direction.turns()]
        
        case _, hops if hops >= 4:
            return [(last_direction, hops_in_this_direction + 1), *[(t, 1) for t in last_direction.turns()]]
        
        case _:
            return [(last_direction, hops_in_this_direction + 1)]

In [44]:
source = Point(0,0)
target = Point(len(graph) - 1, len(graph[0]) - 1)
# target = Point(12, 12)
print(source, target)
dist, prev = dijkstra2(graph, source, target)

Point(y=0, x=0) Point(y=12, x=12)


 33%|███▎      | 48/144 [00:00<00:00, 61455.00it/s]  


In [45]:
s = []
target = Point(len(graph) - 1, len(graph[0]) - 1)
targets = [
    (n, d) for (n, d) in dist.items()
    if n[0] == target
]
target = reduce(lambda p1, p2: p1 if p1[1] < p2[1] else p2, targets)
u, min_dist = target
if prev[u] or u == Point(0, 0):
    while u:
        s.append(u)
        if u in prev:
            u = prev[u]
        else:
            u = None
s = s[::-1]
s

[]